In [4]:
import os
import csv
from operator import index
import math
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import IPython.display as ipd
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize
import warnings
from sklearn.model_selection import train_test_split

plt.rcParams['figure.figsize'] = (12, 8)

sample_rate = 22050
duration = 30  # mesaured in seconds
samples_per_track = sample_rate * duration

In [5]:
def read_dataset_only_metre():
    data = pd.read_csv("../data.csv")
    data = data[data.meter.notnull()]
    return data

In [6]:
def extract_melspectrogram(n_mfcc=13, hop_length=512, n_fft=2048, num_segments=5):
    # dictionary to store the data
    header = 'filename chroma_stft_avg chroma_stft_min chroma_stft_max chroma_stft_mid chroma_stft_std' 
    header+= ' rmse_avg rmse_min rmse_max rmse_mid rmse_std'
    header+= ' spectral_cent_avg spectral_cent_min spectral_cent_max spectral_cent_mid spectral_cent_std'
    header+= ' spectral_band_avg spectral_band_min spectral_band_max spectral_band_mid spectral_band_std'
    header+= ' rolloff_avg rolloff_min rolloff_max rolloff_mid rolloff_std'
    header+= ' zero_crossing_avg zero_crossing_min zero_crossing_max zero_crossing_mid zero_crossing_std'
    for i in range(1, 14):
        header += f' mfcc_avg{i} mfcc_min{i} mfcc_max{i} mfcc_mid{i} mfcc_std{i}'
    header += ' label'
    header = header.split()

    file = open('../multiple_features.csv', 'w', newline='')
    with file:
        writer = csv.writer(file)
        writer.writerow(header)
        
    num_samples_per_segment = int(samples_per_track / num_segments)
    expected_num_vectors_per_segment = math.ceil(num_samples_per_segment / hop_length)
    
    # loop through all songs
    alldata = read_dataset_only_metre()
    
    data = alldata.groupby(['meter'])
    
    for meter, group in data:
        for index, song in group.iterrows():
            path = os.path.relpath("../../Data/genres_original" + song.path)
            signal, sr = librosa.load(path, sr=sample_rate)
            
            for s in range(num_segments):
                start_sample = num_samples_per_segment * s
                finish_sample = start_sample + num_samples_per_segment
                chroma_stft = librosa.feature.chroma_stft(signal[start_sample:finish_sample], sr=sr)
                rmse = librosa.feature.rms(signal[start_sample:finish_sample])
                spec_cent = librosa.feature.spectral_centroid(signal[start_sample:finish_sample], sr=sr)
                spec_bw = librosa.feature.spectral_bandwidth(signal[start_sample:finish_sample], sr=sr)
                rolloff = librosa.feature.spectral_rolloff(signal[start_sample:finish_sample], sr=sr)
                zcr = librosa.feature.zero_crossing_rate(signal[start_sample:finish_sample])
                mfcc = librosa.feature.mfcc(signal[start_sample:finish_sample], n_mfcc=n_mfcc, sr=sr)
                to_append = f'{song.path+str(s)} {np.mean(chroma_stft)} {np.min(chroma_stft)} {np.max(chroma_stft)} {np.median(chroma_stft)} {np.std(chroma_stft)}'
                to_append += f' {np.mean(rmse)} {np.min(rmse)} {np.max(rmse)} {np.median(rmse)} {np.std(rmse)}' 
                to_append += f' {np.mean(spec_cent)} {np.min(spec_cent)} {np.max(spec_cent)} {np.median(spec_cent)} {np.std(spec_cent)} '
                to_append += f' {np.mean(spec_bw)} {np.min(spec_bw)} {np.max(spec_bw)} {np.median(spec_bw)} {np.std(spec_bw)}'
                to_append += f' {np.mean(rolloff)} {np.min(rolloff)} {np.max(rolloff)} {np.median(rolloff)} {np.std(rolloff)}' 
                to_append += f' {np.mean(zcr)} {np.min(zcr)} {np.max(zcr)} {np.median(zcr)} {np.std(zcr)}'    
                for e in mfcc:
                    to_append += f' {np.mean(e)} {np.min(e)} {np.max(e)} {np.median(e)} {np.std(e)}'
                to_append += f' {song.meter}'
                file = open('../multiple_features.csv', 'a', newline='')
                with file:
                    writer = csv.writer(file)
                    writer.writerow(to_append.split())
                
        
extract_melspectrogram(num_segments=3)

/Users/pianistprogrammer/opt/anaconda3/lib/python3.9/site-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")
